<a href="https://colab.research.google.com/github/tiszalab/hybrid-capture-probe-coverage/blob/main/colab_probe_coverage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probe Coverage Analysis for Virus Genomes 🦯 🦠

This notebook analyzes how well sequences from a hybrid-capture probe panel cover a genome of interest.

## **Requirements:**
- Probe sequences (.fasta format)
- Genome sequence of interest (.fasta format)
- [OPTIONAL] Gene feature file for genome (.gff3 format)

**TIP:** Search [NCBI Virus](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/) to find and download genome sequences from viruses you want to check.

## **INSTRUCTIONS**:
On the left edge of each cell below, a Run Button ▶ will appear upon hovering. Starting from the top, click the Run Buttons and follow the prompts.


---


**Example coverage plot:**

<img src="https://github.com/Human-Virome/hybrid-capture-panel-hvp/blob/main/docs/PX361010.1.probes.png?raw=true" height="400" align="bottom" style="height:440px">



In [ ]:
#@title Google Colab Setup - Use Run Button ▶ to install required packages and enable widgets
%pip install -q mappy #polars plotly ipywidgets pandas numpy

# Enable Colab widgets
from google.colab import output
output.enable_custom_widget_manager()

print("✅ Packages installed and widgets enabled!")

In [ ]:
#@title Use Run Button ▶ to generate file upload box
#@markdown Large probe files may take a minute or two to complete upload.
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

# Helper function to handle file upload using google.colab.files.upload
def upload_fasta_file(description):
    print(f"Please upload your {description}:")
    uploaded = files.upload()
    if not uploaded:
        raise ValueError(f"No file uploaded for {description}.")
    filename = list(uploaded.keys())[0]
    temp_path = Path(f"/content/{filename}")
    with open(temp_path, 'wb') as f:
        f.write(uploaded[filename])
    return str(temp_path.resolve())

def upload_gff_file(description):
    print(f"Upload optional {description} or hit CANCEL UPLOAD button:")
    uploaded = files.upload()
    if not uploaded:
        raise ValueError(f"No file uploaded for {description}.")
    filename = list(uploaded.keys())[0]
    temp_path = Path(f"/content/{filename}")
    with open(temp_path, 'wb') as f:
        f.write(uploaded[filename])
    return str(temp_path.resolve())

# Perform file uploads directly when the cell is run
probe_fasta_path = None
genome_fasta_path = None
gff_file_path = None
try:
    probe_fasta_path = upload_fasta_file("Probe FASTA")
    genome_fasta_path = upload_fasta_file("Genome FASTA")
except ValueError as e:
    print(f"❌ File upload error: {e}")
    # Set paths to None to indicate failure, and allow other widgets to display
    probe_fasta_path = None
    genome_fasta_path = None

try:
    gff_file_path = upload_gff_file("Genome gene features table")
except Exception as e:
  print(f"> GFF File not uploaded: {e}")
# GFF file widget (remains as text input for path)
#gff_file_widget = widgets.Text(
#    value='',
##    placeholder='Optional: Path to GFF file for gene annotations (or leave empty if not uploading)',
#    description='GFF File Path:',
#    style={'description_width': 'initial'},
#    layout=widgets.Layout(width='80%')
#)

# Alignment parameters
min_mapq_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=60,
    step=1,
    description='Min MAPQ:',
    style={'description_width': 'initial'}
)

min_aln_len_widget = widgets.IntSlider(
    value=50,
    min=20,
    max=200,
    step=10,
    description='Min Alignment Length:',
    style={'description_width': 'initial'}
)

# Status output
status_output = widgets.Output()

# Store parameters in a dict for downstream use
params = {}

def validate_and_store(btn):
    with status_output:
        clear_output()

        errors = []

        # Use paths obtained from direct upload
        if probe_fasta_path is None:
            errors.append("❌ Probe FASTA file was not uploaded successfully.")
        else:
            params['probe_fasta'] = probe_fasta_path

        if genome_fasta_path is None:
            errors.append("❌ Genome FASTA file was not uploaded successfully.")
        else:
            params['genome_fasta'] = genome_fasta_path

        if gff_file_path is None:
            params['gff_file'] = None
            print("Optional GFF not uploaded")
        else:
            params['gff_file'] = gff_file_path


        if errors:
            for e in errors:
                print(e)
            return

        # Store alignment parameters
        params['min_mapq'] = min_mapq_widget.value
        params['min_aln_len'] = min_aln_len_widget.value

        print("✅ Parameters validated and stored!")
        print(f"   Probe FASTA: {params['probe_fasta']}")
        print(f"   Genome FASTA: {params['genome_fasta']}")
        print(f"   GFF File: {params['gff_file'] or 'None'}")
        print(f"   Min MAPQ: {params['min_mapq']}")
        print(f"   Min Alignment Length: {params['min_aln_len']}")


validate_btn = widgets.Button(description='Validate & Store', button_style='primary')
validate_btn.on_click(validate_and_store)

# Display GUI
print("=== Probe Coverage Analysis Parameters ===")
display(widgets.VBox([
    widgets.HTML("<b>Input Files (Uploads handled above):</b>"),
    widgets.HTML(f"Probe FASTA: {probe_fasta_path if probe_fasta_path else 'Not uploaded'}"),
    widgets.HTML(f"Genome FASTA: {genome_fasta_path if genome_fasta_path else 'Not uploaded'}"),
    widgets.HTML("<br><b>Alignment Filters:</b>"),
    min_mapq_widget,
    min_aln_len_widget,
    widgets.HBox([validate_btn]),
    status_output
]))

In [ ]:
#@title Run to parse input genome
import mappy as mp
import polars as pl

def parse_fasta_simple(fasta_path: str) -> list[tuple[str, str]]:
    """Parse a FASTA file and return list of (name, sequence) tuples."""
    sequences = []
    current_name = None
    current_seq = []

    with open(fasta_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if current_name is not None:
                    sequences.append((current_name, ''.join(current_seq)))
                current_name = line[1:].split()[0]
                current_seq = []
            else:
                current_seq.append(line)

        if current_name is not None:
            sequences.append((current_name, ''.join(current_seq)))

    return sequences

# Load and validate genome (single contig only)
genome_seqs = parse_fasta_simple(params['genome_fasta'])

if len(genome_seqs) != 1:
    raise ValueError(
        f"Expected exactly 1 genome/contig, but found {len(genome_seqs)}. "
        f"Please provide a FASTA file with a single sequence."
    )

genome_name, genome_seq = genome_seqs[0]
genome_length = len(genome_seq)

print(f"Genome loaded: {genome_name}")
print(f"Genome length: {genome_length:,} bp")

In [ ]:
#@title Build mappy index with permissive settings for short probe sequences
#@markdown Using preset 'sr' (short read) for better sensitivity with probes
aligner = mp.Aligner(
    seq=genome_seq,
    preset='sr',
    best_n=5  # Allow multiple alignments per probe
)

if not aligner:
    raise RuntimeError("Failed to build mappy index")

print("Mappy index built successfully")

# Load probes and align
probe_seqs = parse_fasta_simple(params['probe_fasta'])
print(f"Loaded {len(probe_seqs):,} probes")

# Align probes to genome
alignments = []

for probe_name, probe_seq in probe_seqs:
    for hit in aligner.map(probe_seq):
        # Calculate mismatches and gaps from CIGAR
        # NM tag gives edit distance, we need to parse cigar for gaps
        n_gaps = 0
        n_insertions = 0
        n_deletions = 0

        if hit.cigar:
            for length, op in hit.cigar:
                if op == 1:  # Insertion
                    n_insertions += length
                    n_gaps += 1
                elif op == 2:  # Deletion
                    n_deletions += length
                    n_gaps += 1

        # NM is edit distance (mismatches + indel bases)
        # mismatches = NM - insertions - deletions
        n_mismatches = hit.NM - n_insertions - n_deletions

        alignments.append({
            'probe_name': probe_name,
            'probe_length': len(probe_seq),
            'ref_start': hit.r_st,
            'ref_end': hit.r_en,
            'query_start': hit.q_st,
            'query_end': hit.q_en,
            'strand': '+' if hit.strand == 1 else '-',
            'mapq': hit.mapq,
            'alignment_length': hit.r_en - hit.r_st,
            'n_mismatches': n_mismatches,
            'n_gaps': n_gaps,
            'n_insertions': n_insertions,
            'n_deletions': n_deletions,
            'edit_distance': hit.NM,
            'is_primary': hit.is_primary
        })

# Create polars DataFrame
df_alignments = pl.DataFrame(alignments)

print(f"\nTotal alignments: {len(df_alignments):,}")

if len(df_alignments) == 0:
  print(f"\n❌ Since No probe-genome alignments were found, ")
  print(f"❌ downstream analyses will not run. ")
  print(f"❌ Likely, the probes are not a good match to your genome.")

else:
  print(f"Unique probes with alignments: {df_alignments['probe_name'].n_unique():,}")

  # Apply filters
  df_filtered = df_alignments.filter(
      (pl.col('mapq') >= params['min_mapq']) &
      (pl.col('alignment_length') >= params['min_aln_len'])
  )

  print(f"Alignments after filtering: {len(df_filtered):,}")
  df_filtered.head(10)

In [ ]:
#@title Parse GFF file if provided
df_genes = None

if params['gff_file']:
    gff_records = []

    with open(params['gff_file'], 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue

            parts = line.strip().split('\t')
            if len(parts) < 9:
                continue

            seqid, source, feature_type, start, end, score, strand, phase, attributes = parts

            # Parse attributes to get gene name/ID
            attr_dict = {}
            for attr in attributes.split(';'):
                if '=' in attr:
                    key, value = attr.split('=', 1)
                    attr_dict[key.strip()] = value.strip()

            # Get a display name (prefer Name, then gene, then ID)
            name = attr_dict.get('Name', attr_dict.get('gene', attr_dict.get('ID', '')))

            gff_records.append({
                'seqid': seqid,
                'source': source,
                'feature_type': feature_type,
                'start': int(start),
                'end': int(end),
                'strand': strand,
                'name': name
            })

    df_genes = pl.DataFrame(gff_records)

    # Filter to gene/CDS features for display
    df_genes_display = df_genes.filter(
        pl.col('feature_type').is_in(['gene'])
    )

    print(f"Loaded {len(df_genes):,} GFF records")
    print(f"Gene/CDS features: {len(df_genes_display):,}")
    display(df_genes_display.head(10))
else:
    print("No GFF file provided - skipping gene annotations")

In [ ]:
#@title Plot probe alignments onto genome
import numpy as np
import pandas as pd

df_plot = df_filtered.with_columns([
    # Combined quality metric: gaps + mismatches
    (pl.col('n_gaps') + pl.col('n_mismatches')).alias('total_errors'),
]).to_pandas()

# Assign y positions to avoid overlaps (greedy interval scheduling)
df_plot = df_plot.sort_values('ref_start').reset_index(drop=True)

y_positions = []
track_ends = {}  # track_id -> end_position

for idx, row in df_plot.iterrows():
    start = row['ref_start']
    end = row['ref_end']

    # Find first available track
    assigned = False
    for track_id in sorted(track_ends.keys()):
        if track_ends[track_id] < start:
            y_positions.append(track_id)
            track_ends[track_id] = end
            assigned = True
            break

    if not assigned:
        new_track = len(track_ends)
        y_positions.append(new_track)
        track_ends[new_track] = end

df_plot['y_pos'] = y_positions
n_tracks = max(y_positions) + 1 if y_positions else 1

print(f"Alignments arranged in {n_tracks} tracks")

# Interactive probe coverage plot using Plotly
import plotly.graph_objects as go

# Create hover text with probe details
hover_text = [
    f"<b>{row['probe_name']}</b><br>"
    f"Position: {row['ref_start']:,} - {row['ref_end']:,}<br>"
    f"Length: {row['alignment_length']:,} bp<br>"
    f"Strand: {row['strand']}<br>"
    f"Errors: {row['total_errors']} (gaps: {row['n_gaps']}, mismatches: {row['n_mismatches']})<br>"
    f"MAPQ: {row['mapq']}"
    for _, row in df_plot.iterrows()
]

# Create the figure
fig = go.Figure()

# Add each alignment as a separate trace for proper hover
for idx, row in df_plot.iterrows():
    max_errors = df_plot['total_errors'].max() if df_plot['total_errors'].max() > 0 else 1
    error_ratio = row['total_errors'] / max_errors

    # Color interpolation: blue (#2166ac) -> white (#f7f7f7) -> red (#b2182b)
    if error_ratio <= 0.5:
        r = int(33 + (247 - 33) * (error_ratio * 2))
        g = int(102 + (247 - 102) * (error_ratio * 2))
        b = int(172 + (247 - 172) * (error_ratio * 2))
    else:
        r = int(247 + (178 - 247) * ((error_ratio - 0.5) * 2))
        g = int(247 + (24 - 247) * ((error_ratio - 0.5) * 2))
        b = int(247 + (43 - 247) * ((error_ratio - 0.5) * 2))

    color = f'rgb({r},{g},{b})'

    fig.add_trace(go.Scatter(
        x=[row['ref_start'], row['ref_end']],
        y=[row['y_pos'], row['y_pos']],
        mode='lines',
        line=dict(color=color, width=6),
        hoverinfo='text',
        hovertext=hover_text[idx],
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title=dict(
        text=f'<b>Probe Coverage of {genome_name}</b><br><sup>{len(df_plot):,} alignments from {df_plot["probe_name"].nunique():,} probes</sup>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Genome Position (bp)',
        range=[0, genome_length],
        tickformat=','
    ),
    yaxis=dict(
        title='',
        showticklabels=False,
        showgrid=False,
        zeroline=False
    ),
    plot_bgcolor='white',
    height=max(400, n_tracks * 30),
    hovermode='closest'
)

fig.show()

In [ ]:
#@title Plot probe alignments and gene coordinates from GFF
#@markdown This will only run if you provided a GFF at the beginning
import plotly.graph_objects as go

if df_genes is not None:
    import pandas as pd

    # Filter to CDS/gene features
    df_genes_plot = df_genes_display.to_pandas()

    if len(df_genes_plot) > 0:
        # Create the figure
        fig_genes = go.Figure()

        # Position genes below the alignments
        gene_y = -2

        # Add gene annotations as filled rectangles
        for _, gene in df_genes_plot.iterrows():
            fig_genes.add_shape(
                type="rect",
                x0=gene['start'],
                x1=gene['end'],
                y0=gene_y - 0.8,
                y1=gene_y + 0.8,
                fillcolor='rgba(77, 175, 74, 0.6)',
                line=dict(color='#2d6a2e', width=1)
            )

            # Add gene label
            if gene['name']:
                fig_genes.add_annotation(
                    x=gene['start'] + 50,
                    y=gene_y,
                    text=f"<b>{gene['name']}</b>",
                    showarrow=False,
                    font=dict(size=10, color='black'),
                    xanchor='left'
                )

        # Add invisible trace for gene hover info
        for _, gene in df_genes_plot.iterrows():
            fig_genes.add_trace(go.Scatter(
                x=[(gene['start'] + gene['end']) / 2],
                y=[gene_y],
                mode='markers',
                marker=dict(size=0.1, opacity=0),
                hoverinfo='text',
                hovertext=f"<b>Gene: {gene['name']}</b><br>"
                         f"Position: {gene['start']:,} - {gene['end']:,}<br>"
                         f"Length: {gene['end'] - gene['start']:,} bp<br>"
                         f"Strand: {gene['strand']}",
                showlegend=False
            ))

        # Add probe alignments
        for idx, row in df_plot.iterrows():
            max_errors = df_plot['total_errors'].max() if df_plot['total_errors'].max() > 0 else 1
            error_ratio = row['total_errors'] / max_errors

            # Color interpolation: blue -> white -> red
            if error_ratio <= 0.5:
                r = int(33 + (247 - 33) * (error_ratio * 2))
                g = int(102 + (247 - 102) * (error_ratio * 2))
                b = int(172 + (247 - 172) * (error_ratio * 2))
            else:
                r = int(247 + (178 - 247) * ((error_ratio - 0.5) * 2))
                g = int(247 + (24 - 247) * ((error_ratio - 0.5) * 2))
                b = int(247 + (43 - 247) * ((error_ratio - 0.5) * 2))

            color = f'rgb({r},{g},{b})'

            hover_text = (
                f"<b>{row['probe_name']}</b><br>"
                f"Position: {row['ref_start']:,} - {row['ref_end']:,}<br>"
                f"Length: {row['alignment_length']:,} bp<br>"
                f"Strand: {row['strand']}<br>"
                f"Errors: {row['total_errors']} (gaps: {row['n_gaps']}, mismatches: {row['n_mismatches']})<br>"
                f"MAPQ: {row['mapq']}"
            )

            fig_genes.add_trace(go.Scatter(
                x=[row['ref_start'], row['ref_end']],
                y=[row['y_pos'], row['y_pos']],
                mode='lines',
                line=dict(color=color, width=6),
                hoverinfo='text',
                hovertext=hover_text,
                showlegend=False
            ))

        # Update layout
        fig_genes.update_layout(
            title=dict(
                text=f'<b>Probe Coverage of {genome_name} with Gene Annotations</b><br>'
                     f'<sup>{len(df_plot):,} alignments | Green bars = gene features</sup>',
                x=0.5,
                xanchor='center'
            ),
            xaxis=dict(
                title='Genome Position (bp)',
                range=[0, genome_length],
                tickformat=','
            ),
            yaxis=dict(
                title='',
                showticklabels=False,
                showgrid=False,
                zeroline=False,
                range=[gene_y - 1.5, n_tracks + 0.5]
            ),
            plot_bgcolor='white',
            height=max(450, n_tracks * 30 + 100),
            hovermode='closest'
        )

        fig_genes.show()
    else:
        print("No gene/CDS features found in GFF file")
else:
    print("No GFF file provided - skipping gene annotation plot")

In [ ]:
#@title Probe Shoulder Length for soft coverage calculation
#@markdown since library fragments are longer than probes, we can expect that "shoulder" regions adjacent to probes will be covered as well.
#@markdown 50bp is a very safe assumption.
import ipywidgets as widgets
from IPython.display import display

shoulder_length_widget = widgets.IntSlider(
    value=50,
    min=0,
    max=200,
    step=10,
    description='Probe Shoulder Length:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display(widgets.VBox([
    widgets.HTML("<b>Soft Coverage Settings</b>"),
    widgets.HTML("<p style='color: gray;'>Extends probe alignments by this many nucleotides on each side to estimate potential capture regions.</p>"),
    shoulder_length_widget
]))

In [ ]:
#@title Calculate coverage statistics (with and without soft coverage)
import numpy as np

# Get shoulder length from widget
shoulder_length = shoulder_length_widget.value

# Create coverage arrays
coverage = np.zeros(genome_length, dtype=int)
soft_coverage = np.zeros(genome_length, dtype=int)

for _, row in df_plot.iterrows():
    start = int(row['ref_start'])
    end = int(row['ref_end'])

    # Regular coverage
    coverage[start:end] += 1

    # Soft coverage (extended by shoulder_length on each side)
    soft_start = max(0, start - shoulder_length)
    soft_end = min(genome_length, end + shoulder_length)
    soft_coverage[soft_start:soft_end] += 1

# Calculate statistics for regular coverage
bases_covered = np.sum(coverage > 0)
pct_1x = 100 * np.sum(coverage >= 1) / genome_length
pct_2x = 100 * np.sum(coverage >= 2) / genome_length
pct_5x = 100 * np.sum(coverage >= 5) / genome_length
mean_depth = np.mean(coverage)
median_depth = np.median(coverage)
max_depth = np.max(coverage)

# Calculate statistics for soft coverage
soft_bases_covered = np.sum(soft_coverage > 0)
soft_pct_1x = 100 * np.sum(soft_coverage >= 1) / genome_length
soft_pct_2x = 100 * np.sum(soft_coverage >= 2) / genome_length
soft_pct_5x = 100 * np.sum(soft_coverage >= 5) / genome_length
soft_mean_depth = np.mean(soft_coverage)
soft_median_depth = np.median(soft_coverage)
soft_max_depth = np.max(soft_coverage)

print("=" * 70)
print("COVERAGE SUMMARY")
print("=" * 70)
print(f"Genome: {genome_name}")
print(f"Genome length: {genome_length:,} bp")
print(f"Total probe alignments: {len(df_plot):,}")
print(f"Probe shoulder length: {shoulder_length} bp")
print()

# Side-by-side comparison
print(f"{'Metric':<30} {'Regular':>15} {'Soft Coverage':>15}")
print("-" * 70)
print(f"{'Bases covered (≥1x)':<30} {bases_covered:>12,} bp {soft_bases_covered:>12,} bp")
print(f"{'% covered (≥1x)':<30} {pct_1x:>14.1f}% {soft_pct_1x:>14.1f}%")
print(f"{'Bases covered (≥2x)':<30} {np.sum(coverage >= 2):>12,} bp {np.sum(soft_coverage >= 2):>12,} bp")
print(f"{'% covered (≥2x)':<30} {pct_2x:>14.1f}% {soft_pct_2x:>14.1f}%")
print(f"{'Bases covered (≥5x)':<30} {np.sum(coverage >= 5):>12,} bp {np.sum(soft_coverage >= 5):>12,} bp")
print(f"{'% covered (≥5x)':<30} {pct_5x:>14.1f}% {soft_pct_5x:>14.1f}%")
print()
print(f"{'Mean coverage depth':<30} {mean_depth:>14.2f}x {soft_mean_depth:>14.2f}x")
print(f"{'Median coverage depth':<30} {median_depth:>14.1f}x {soft_median_depth:>14.1f}x")
print(f"{'Max coverage depth':<30} {max_depth:>15}x {soft_max_depth:>15}x")
print("=" * 70)

# Alignment quality summary
print("\nALIGNMENT QUALITY")
print("-" * 70)
print(f"Alignments with 0 errors: {len(df_plot[df_plot['total_errors'] == 0]):,}")
print(f"Alignments with 1-2 errors: {len(df_plot[(df_plot['total_errors'] >= 1) & (df_plot['total_errors'] <= 2)]):,}")
print(f"Alignments with 3+ errors: {len(df_plot[df_plot['total_errors'] >= 3]):,}")
print(f"Mean errors per alignment: {df_plot['total_errors'].mean():.2f}")

In [ ]:
#@title Plot coverage depth across the genome
import plotly.graph_objects as go
import pandas as pd

# Bin the coverage for smoother plotting
bin_size = max(1, genome_length // 1000)
n_bins = genome_length // bin_size

binned_coverage = []
for i in range(n_bins):
    start = i * bin_size
    end = min((i + 1) * bin_size, genome_length)
    binned_coverage.append({
        'position': (start + end) // 2,
        'depth': np.mean(coverage[start:end])
    })

df_depth = pd.DataFrame(binned_coverage)

# Create the figure
fig_depth = go.Figure()

# Add bar chart for coverage depth
fig_depth.add_trace(go.Bar(
    x=df_depth['position'],
    y=df_depth['depth'],
    marker_color='#2166ac',
    opacity=0.7,
    hovertemplate='Position: %{x:,}<br>Depth: %{y:.1f}x<extra></extra>'
))

# Update layout
fig_depth.update_layout(
    title=dict(
        text=f'<b>Coverage Depth Across {genome_name}</b><br><sup>Bin size: {bin_size:,} bp | Mean depth: {mean_depth:.1f}x</sup>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Genome Position (bp)',
        range=[0, genome_length],
        tickformat=','
    ),
    yaxis=dict(
        title='Coverage Depth'
    ),
    plot_bgcolor='white',
    height=400,
    bargap=0
)

fig_depth.show()

In [ ]:
#@title Plot histogram of gap lengths (regions with no probe coverage)
import plotly.graph_objects as go

# Find gaps (consecutive regions of zero coverage)
gap_lengths = []
in_gap = False
gap_start = 0

for i, depth in enumerate(coverage):
    if depth == 0:
        if not in_gap:
            in_gap = True
            gap_start = i
    else:
        if in_gap:
            gap_lengths.append(i - gap_start)
            in_gap = False

# Handle gap at end of genome
if in_gap:
    gap_lengths.append(len(coverage) - gap_start)

# Summary statistics
n_gaps = len(gap_lengths)
total_gap_bp = sum(gap_lengths)
mean_gap = np.mean(gap_lengths) if gap_lengths else 0
median_gap = np.median(gap_lengths) if gap_lengths else 0
max_gap = max(gap_lengths) if gap_lengths else 0

print("=" * 50)
print("GAP ANALYSIS (Regions with 0x coverage)")
print("=" * 50)
print(f"Number of gaps: {n_gaps:,}")
print(f"Total gap bases: {total_gap_bp:,} bp ({100*total_gap_bp/genome_length:.1f}% of genome)")
print(f"Mean gap length: {mean_gap:.1f} bp")
print(f"Median gap length: {median_gap:.1f} bp")
print(f"Max gap length: {max_gap:,} bp")
print("=" * 50)

# Plot histogram
if gap_lengths:
    fig_gaps = go.Figure()

    # Add histogram
    fig_gaps.add_trace(go.Histogram(
        x=gap_lengths,
        nbinsx=30,
        marker_color='#d62728',
        opacity=0.7,
        hovertemplate='Gap length: %{x} bp<br>Count: %{y}<extra></extra>'
    ))

    # Add mean line
    fig_gaps.add_vline(
        x=mean_gap,
        line_dash="dash",
        line_color="black",
        line_width=2,
        annotation_text=f"Mean: {mean_gap:.0f} bp",
        annotation_position="top"
    )

    # Update layout
    fig_gaps.update_layout(
        title=dict(
            text=f'<b>Distribution of Probe Coverage Gaps in {genome_name}</b><br><sup>{n_gaps:,} gaps | Mean: {mean_gap:.0f} bp | Max: {max_gap:,} bp</sup>',
            x=0.5,
            xanchor='center'
        ),
        xaxis=dict(
            title='Gap Length (bp)'
        ),
        yaxis=dict(
            title='Count'
        ),
        plot_bgcolor='white',
        height=400
    )

    fig_gaps.show()
else:
    print("No gaps found - genome has complete probe coverage!")

In [ ]:
#@title Download probe sequences that align to this genome
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, FileLink
from google.colab import files

# Get unique probe names with alignments
aligned_probe_names = set(df_filtered['probe_name'].to_list())
print(f"Found {len(aligned_probe_names):,} unique probes with alignments to {genome_name}")

# Filter original probe sequences to only those with alignments
aligned_probes = [(name, seq) for name, seq in probe_seqs if name in aligned_probe_names]
print(f"Filtered {len(aligned_probes):,} probe sequences")

# Output file path widget
default_output = f"{genome_name}_aligned_probes.fasta"

output_path_widget = widgets.Text(
    value=default_output,
    description='Output FASTA:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

save_output = widgets.Output()

def save_probes(btn):
    with save_output:
        save_output.clear_output()
        output_filename = output_path_widget.value

        try:
            # Save to local Colab storage first
            with open(output_filename, 'w') as f:
                for name, seq in aligned_probes:
                    f.write(f">{name}\n{seq}\n")

            print(f"✅ Saved {len(aligned_probes):,} probe sequences")
            print(f"\nDownloading file...")

            # Trigger download in Colab
            files.download(output_filename)
        except Exception as e:
            print(f"❌ Error saving file: {e}")

save_btn = widgets.Button(description='Save & Download FASTA', button_style='success', icon='download')
save_btn.on_click(save_probes)

display(widgets.VBox([
    widgets.HTML("<h3>Export Aligned Probe Sequences</h3>"),
    widgets.HTML(f"<p>{len(aligned_probes):,} probes aligned to <b>{genome_name}</b></p>"),
    output_path_widget,
    save_btn,
    save_output
]))